In [28]:
from os import listdir
from os.path import isfile, join
import string
import nltk
import os
import re
import string
import pandas as pd
import numpy as np
from pathlib import PurePath,PureWindowsPath
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from itertools import chain
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
import pickle
import joblib
warnings.filterwarnings("ignore")

In [29]:
def remove_metadata(lines):
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines

In [30]:
# Remove the html in sample text
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)


In [31]:
def lemmaLower(words):
    words = words.split()
    #we also normalize the cases of our words
    words = [word.lower() for word in words]
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    #removing stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # joining splited words to be one string per raw again
    joined_string = ' '.join(words)
    # delete extra spaces
    joined_string = re.sub (r'\b\w\b','', joined_string)
    
    return joined_string

In [32]:
def prepare(path):
    #read data from file of the given path
    f = open(path, 'r')
    text_lines = f.readlines()
    
    #removing the meta-data at the top of each document
    text_lines = remove_metadata(text_lines)
    
    #initiazing an array to hold all the words in a document
    doc_words = []
    
    # add each file as a row in the list 
    doc_words.append(text_lines)
    
    return doc_words

In [33]:
def Text_Cleaning(data):
    
    # replace \n and \t in files by empty strings
    data= data.replace('\\n','')
    data= data.replace('\\t','')
    
    #replace any non-charachter by space
    data  = re.sub('[^a-zA-Z]', ' ', data)
  
    #remove html tages
    data = remove_html(data)
    
    #remove punctuation
    punctuationfree = "".join([i for i in data if i not in string.punctuation])
    data = punctuationfree
    
    # remove any extra spaces
    data = [re.sub (r'\b\w\b','', data)]
    
    return data

In [34]:
cwd = os.getcwd()
my_path = r"C:\Users\future\Desktop\20_newsgroups"
os.chdir(my_path)
cwd = os.getcwd()
#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]
folders


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [35]:
files = []
# read each file in each folder and append it to files list
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])
files

[['49960',
  '51060',
  '51119',
  '51120',
  '51121',
  '51122',
  '51123',
  '51124',
  '51125',
  '51126',
  '51127',
  '51128',
  '51129',
  '51130',
  '51131',
  '51132',
  '51133',
  '51134',
  '51135',
  '51136',
  '51137',
  '51138',
  '51139',
  '51140',
  '51141',
  '51142',
  '51143',
  '51144',
  '51145',
  '51146',
  '51147',
  '51148',
  '51149',
  '51150',
  '51151',
  '51152',
  '51153',
  '51154',
  '51155',
  '51156',
  '51157',
  '51158',
  '51159',
  '51160',
  '51161',
  '51162',
  '51163',
  '51164',
  '51165',
  '51166',
  '51167',
  '51168',
  '51169',
  '51170',
  '51171',
  '51172',
  '51173',
  '51174',
  '51175',
  '51176',
  '51177',
  '51178',
  '51179',
  '51180',
  '51181',
  '51182',
  '51183',
  '51184',
  '51185',
  '51186',
  '51187',
  '51188',
  '51189',
  '51190',
  '51191',
  '51192',
  '51193',
  '51194',
  '51195',
  '51196',
  '51197',
  '51198',
  '51199',
  '51200',
  '51201',
  '51202',
  '51203',
  '51204',
  '51205',
  '51206',
  '51207',

In [36]:
#creating a list of pathnames of all the documents
pathname_list = []
for i in range(len(folders)):
    for f in files[i]:
        pathname_list.append(join(my_path, join(folders[i], f)))

pathname_list

['C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\49960',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51060',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51119',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51120',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51121',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51122',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51123',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51124',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51125',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51126',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51127',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51128',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51129',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51130',
 'C:\\Users\\future\\Desktop\\20_newsgroups\\alt.atheism\\51131',
 'C:\\User

In [37]:
len(pathname_list)

19997

In [38]:
#making an array containing the classes each of the documents belong to

Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)
        
Y

['alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.atheism',
 'alt.at

In [39]:
len(Y)

19997

In [42]:
# Creating X
list_of_words = []

for document in pathname_list:
        list_of_words.append(prepare(str(document)))

list_of_words

[[['Archive-name: atheism/resources\n',
   'Alt-atheism-archive-name: resources\n',
   'Last-modified: 11 December 1992\n',
   'Version: 1.0\n',
   '\n',
   '                              Atheist Resources\n',
   '\n',
   '                      Addresses of Atheist Organizations\n',
   '\n',
   '                                     USA\n',
   '\n',
   'FREEDOM FROM RELIGION FOUNDATION\n',
   '\n',
   'Darwin fish bumper stickers and assorted other atheist paraphernalia are\n',
   'available from the Freedom From Religion Foundation in the US.\n',
   '\n',
   'Write to:  FFRF, P.O. Box 750, Madison, WI 53701.\n',
   'Telephone: (608) 256-8900\n',
   '\n',
   'EVOLUTION DESIGNS\n',
   '\n',
   'Evolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones\n',
   'Christians stick on their cars, but with feet and the word "Darwin" written\n',
   'inside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.\n',
   '\n',
   'Write to:  Evolution Designs, 7119 Lau

In [54]:
# covert it into dataframe and create Y
df = pd.DataFrame(list_of_words, columns=['documents'])
df['category'] = Y
df.head()

,documents,category
0,"[Archive-name: atheism/resources\n, Alt-atheis...",alt.atheism
1,"[Archive-name: atheism/introduction\n, Alt-ath...",alt.atheism
2,"[In article <65974@mimsy.umd.edu>\n, mangoe@cs...",alt.atheism
3,[dmn@kepler.unh.edu (...until kings become phi...,alt.atheism
4,[In article <N4HY.93Apr5120934@harder.ccr-p.id...,alt.atheism


In [55]:
#Text Cleaning
df["documents"] = df["documents"].apply(lambda x: Text_Cleaning(str(x)))
#remove extra spaces in text
df["documents"] = df["documents"].apply(lambda x: [" ".join(text.split()) for text in x])
df["documents"] = df["documents"].apply(lambda x: " ".join(x))
df.head()

,documents,category
0,Archive name atheism resources Alt atheism arc...,alt.atheism
1,Archive name atheism introduction Alt atheism ...,alt.atheism
2,In article mimsy umd edu mangoe cs umd edu Cha...,alt.atheism
3,dmn kepler unh edu until kings become philosop...,alt.atheism
4,In article HY Apr harder ccr ida org hy harder...,alt.atheism


In [56]:
#spiliting data into train & test
#we use a sample of 5000 raw as the data is so large
X_train, X_test, y_train, y_test = train_test_split(df['documents'][:5000], df['category'][:5000], random_state=0, test_size=0.20)
len(X_train)

4000

In [57]:
# applying TF-IDF
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english',preprocessor = lemmaLower , max_features=55900 , min_df = 0.0001, max_df = 0.97)
tfidfvectorizer.fit(X_train)
X_train_tfidf = tfidfvectorizer.transform(X_train).toarray()
X_test_tfidf = tfidfvectorizer.transform(X_test).toarray()

In [58]:
X_test_tfidf.shape

(1000, 45075)

In [60]:
#Naive Bayes model

gnb = GaussianNB()  
y_pred = gnb.fit(X_train_tfidf, y_train).predict(X_test_tfidf)

print("Naive Bayes Accuracy: ", gnb.score( X_test_tfidf, y_test)*100)

Naive Bayes Accuracy:  82.69999999999999


In [27]:
#SVM model

svm = SVC(kernel='linear')
svm.fit(X_train_tfidf, y_train)
accuracy = svm.score(X_test_tfidf, y_test)

print(f"SVM Accuracy: {accuracy*100}")

SVM Accuracy: 89.2


In [61]:
#KNN model

knn = KNeighborsClassifier(n_neighbors=1500)  # Specify the number of neighbors (K)
knn.fit(X_train_tfidf, y_train)
accuracy_knn = knn.score(X_test_tfidf, y_test)

print(f"KNN Accuracy: {accuracy_knn*100}")

KNN Accuracy: 74.6


In [62]:
# DecisionTree model

dtc = DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=3 , min_samples_leaf=5)
dtc = dtc.fit(X_train_tfidf,y_train)
y_pred = dtc.predict(X_test_tfidf)

print("Decision tree Accuracy:",accuracy_score(y_test, y_pred)*100)

Decision tree Accuracy: 43.0


In [63]:
#RandomForest model

rdf = RandomForestClassifier(n_estimators=100, random_state=42)
rdf.fit(X_train_tfidf, y_train)
y_pred = rdf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100}")

Accuracy: 86.3


In [64]:
#Logistic Regression

log=LogisticRegression(random_state=16)
log.fit(X_train_tfidf,y_train)
x_test_pred=log.predict(X_test_tfidf)
test_accuracy=accuracy_score(x_test_pred,y_test)

print("accuracy of testing data by Logistic Regression is : ",test_accuracy*100)

accuracy of testing data by Logistic Regression is :  89.0
